In [1]:
import pandas as pd

from src.pipelines import pipeline_data_collection, pipeline_data_processing

pd.set_option('display.max_columns', 50)

# Data collection

In [ ]:
update_tables = dict(despesas=True, 
                     deputados=False)
id_deputados_to_fetch = [220526, 178873, 220527, 220530, 160517, 
                         220528, 204356, 204355, 178871, 220529]
start_date = '2024-05-01'
end_date = '2024-05-01'

pipeline_data_collection(update_tables, id_deputados_to_fetch, start_date, end_date)

# Data preprocessing

In [ ]:
update_tables = dict(despesas=True, 
                     deputados=False)

pipeline_data_processing(update_tables)

# tests

In [ ]:
df = pd.read_parquet('data/gold/master_table.parquet')
df_mes = pd.read_parquet('data/gold/monthly_data.parquet')

In [ ]:
df.head(1)

In [ ]:
df_mes

## gasto medio por deputado

In [ ]:
periodos = ['2024_05']

df_filtrado = df.loc[df['ano_mes'].isin(periodos)].copy()
mensal_filtrado = df_mes.loc[df_mes['ano_mes'].isin(periodos)].copy()

In [ ]:
def calc_avg_spend_from_period(df):
    spend = df.groupby('ano_mes')['Valor'].mean().mean()
    return f"R$ {round(spend, 2)}"

calc_avg_spend_from_period(mensal_filtrado)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

## grafico por deputado e tipo de despesa

In [ ]:
px.bar(df.loc[df['ano_mes']=='2024_04'], x='nome', y='valorDocumento', color='tipoDespesa')

## deputado que mais gastou por tipo de despesa

In [ ]:
mais_gastou_por_tipo = dict()
tmp = df_filtrado.groupby(['nome', 'tipoDespesa'])['Valor'].sum().reset_index().copy()
for tipo, tmpdf in tmp.groupby('tipoDespesa'):
    result_tipo = tmpdf.loc[tmpdf['Valor']==tmpdf['Valor'].max(), :].copy()
    mais_gastou_por_tipo[tipo] = dict(Deputado=result_tipo['nome'].values[0], 
                                        Valor = result_tipo['Valor'].values[0])
    # tmp[tmp==tmp.max()]:

mais_gastou_por_tipo = pd.DataFrame(mais_gastou_por_tipo).T.sort_values('Valor', ascending=False)
mais_gastou_por_tipo

In [ ]:
mais_gastou_por_tipo = dict()
tmp = df_filtrado.groupby(['nome', 'tipoDespesa'])['Valor'].sum().reset_index().copy()
for tipo_despesa in tmp['tipoDespesa'].unique():
    result = tmp.loc[tmp['tipoDespesa']==tipo_despesa].copy()
    result = result.loc[result['Valor']==result['Valor'].max(), :].copy()
    mais_gastou_por_tipo[tipo_despesa] = dict(Deputado=result['nome'].values[0], 
                                        Valor = result['Valor'].values[0])
    # tmp[tmp==tmp.max()]:


In [ ]:
pd.DataFrame(mais_gastou_por_tipo).T

In [ ]:
for deputado, tmpdf in df_filtrado.groupby('deputado'):
    tmpdf.loc[tmpdf['Valor']]


In [ ]:
df.groupby('nomeFornecedor')['valorDocumento'].sum().sort_values(ascending=False).head(10)

In [ ]:
df.loc[(df['nome'].str.contains('Linhalis')) & (df['ano_mes']=='2023_12') & (df['valorDocumento'].between(10000, 13000)), ['urlDocumento']].values

In [ ]:
px.line(tmp, x='ano_mes', y='valorDocumento', color='nome')

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
tmp = df.groupby(['id_deputado', 'ano_mes'])[['valorDocumento']].sum()

fig = go.Figure()
for k, tmpdf in df.groupby(['id_deputado']):
    px.line(tmp.loc[dep], y='valorDocumento')
    # fig.add_trace(px.line(tmp.loc[dep], y='valorDocumento'))
fig.show()

# Rascunho

In [ ]:
def pipeline_data_collection():
    info_datasets = {} # config
    reprocessing_status = {}
    for dataset in info_datasets.keys():
        df = collect_data(dataset)
        reprocessing_status[dataset] = get_reprocessing_status()
        if reprocessing_status[dataset]:
            save_data(df)

    save_data(reprocessing_status)

In [ ]:
def pipeline_data_processing():
    
    process_deputados()
    process_despesas()

In [ ]:
pipeline_data_collection()

pipeline_data_processing()

update_website()